# 6-3: 3D Max/Avg Pooling
- Channel-wise(채널별)
- 결과가 Channel은 동일하나, Width, Height가 달라질 수 있음
- 가우스식도 필요함

## Code.6-3-1: 3D Max Pooling

In [12]:
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import MaxPooling2D

N, n_H, n_W, n_C = 1, 5, 5, 3
f, s = 2,2

# 채녈별로 편히 보기 위해 trasnpose

x = tf.random.normal(shape=(N, n_H, n_W, n_C))
# 채널별 4,5의 결과 출력
# print(f"x: {x.shape}\n {np.transpose(x.numpy().squeeze(), (2,0,1))}")

pool_max = MaxPooling2D(pool_size=f, strides=s)
pooled_max = pool_max(x)

pooled_max_t = np.transpose(pooled_max.numpy().squeeze(), (2,0,1))
print(f"pooled_max(Tensorflow): {pooled_max.shape}\n{pooled_max_t}")

x = x.numpy().squeeze()
n_H_ = math.floor((n_H-f)/s + 1)
n_W_ = math.floor((n_W-f)/s + 1)
pooled_max_man = np.zeros(shape=(n_H_, n_W_, n_C))
print(pooled_max_man.shape)

for c in range(n_C):
  c_image = x[:,:,c]
  # s(strides)를 통해 점프
  # 수학적인 내용과 코딩적인 측면의 인덱스 오차를 표현하기 위해 별도의 index 변수 선언
  # stride를 통해 건너뛰는 변수와, pooled_max_man에 할당되는 차이 발생
  h_ = 0
  for h in range(0, n_H -f + 1, s):
    w_ = 0
    for w in range(0, n_W -f + 1, s):
      window = c_image[h:h+f, w:w+f]
      pooled_max_man[h_, w_, c] = np.max(window)

      w_ +=1
    h_ +=1

pooled_max_t = np.transpose(pooled_max_man, (2,0,1))
print(f"pooled_max(Manual): {pooled_max_man.shape}\n{pooled_max_t}")


pooled_max(Tensorflow): (1, 2, 2, 3)
[[[0.74605805 0.99293303]
  [0.1007857  1.466899  ]]

 [[0.46927193 0.91023743]
  [1.6741854  0.38664454]]

 [[0.06659878 0.52235645]
  [1.9468899  1.799589  ]]]
(2, 2, 3)
pooled_max(Manual): (2, 2, 3)
[[[0.74605805 0.99293303]
  [0.1007857  1.46689904]]

 [[0.46927193 0.91023743]
  [1.6741854  0.38664454]]

 [[0.06659878 0.52235645]
  [1.94688988 1.79958904]]]


# 6-4: Padding

## Code.6-4-1: ZeroPadding2D Layer

In [21]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import ZeroPadding2D

images = tf.random.normal(shape=(1, 28, 28, 3))
print(images.shape)
# print(np.transpose(images.numpy().squeeze(), (2,0,1)))

zero_padding = ZeroPadding2D(padding=2)
y = zero_padding(images)
# print(np.transpose(y.numpy().squeeze(), (2,0,1)))
print(y.shape)

(1, 28, 28, 3)
(1, 32, 32, 3)


## Code.6-4-2: Zero Padding with Conv2D Layers

In [25]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D

images = tf.random.normal(shape=(1,28,28,3))
# Conv2D내에서 바로 padding 사용 가능
# padding=valid; 패딩을 안하겠다는 의미
# padding=same; 커널이 움직일때 corner case를 계산하여 사이즈가 줄어들이 않도록 동작
conv = Conv2D(filters=1, kernel_size=3, padding='same')

# 아래와 같이 특정 숫자 기입시 에러 발생
# 결과값이 동일해야하기 때문
# conv = Conv2D(filters=1, kernel_size=3, padding=3)
y = conv(images)
print(y.shape)

(1, 28, 28, 1)


# 6-5: Strides

## Code.6-5-1. Strides in Conv2D Layers

In [27]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D

images = tf.random.normal(shape=(1,28,28,3))
# padding=valid; 패딩을 안하겠다는 의미
conv = Conv2D(filters=1, kernel_size=3, padding='valid', strides=2)
y = conv(images)

print(images.shape)
# strides를 통한 점프, 사이즈가 줄어듦
print(y.shape)

(1, 28, 28, 3)
(1, 13, 13, 1)


## Code.6-5-2. Strides in Pooling Layers

In [34]:
import tensorflow as tf
from tensorflow.keras.layers import MaxPooling2D

images = tf.random.normal(shape=(1,28,28,3))
# padding=valid; 패딩을 안하겠다는 의미
pool = MaxPooling2D(pool_size=3, padding='valid', strides=2)
y = pool(images)

print(images.shape)
# strides를 통한 점프, 사이즈가 줄어듦
# 채널마다 maxPooling을 적용하기때문에, channel size는 유지
# H,W가 동일한 값
print(y.shape)

(1, 28, 28, 3)
(1, 13, 13, 3)
